In [1]:
import gym
import numpy as np

import time

In [2]:
import sys
import os

sys.path.append(os.path.abspath('../'))

del sys, os

In [3]:
import matplotlib.pyplot as plt

In [4]:
# LaTeX rendering in graphs
from distutils.spawn import find_executable
if find_executable('latex'):
    plt.rc('text', usetex=True)

plt.rc('font', family='serif')

# High resolution graphs
%config InlineBackend.figure_format = 'retina'

In [5]:
import torch

In [6]:
%reload_ext autoreload
%autoreload 2

In [7]:
import models.rnn as rnns
import models.mlp as mlps
import control.agents as agents
import control.environments as environments

In [8]:
import copy

# Setup

In [18]:
model = mlps.MLP()
# model = rnns.RNN(input_dimension=128, hidden_dimension=50, truncate=10)  #, key_dimension=15)

In [19]:
optimiser = torch.optim.Adam(model.parameters(), lr=.001)

In [20]:
agent = agents.DQNAgent(model, optimiser)

In [27]:
environment = environments.ExpectedSarsa(
    environment=gym.make('Breakout-ram-v0'), 
    agent=agent, 
    gamma=.999,
    temperature=10,
    verbose=True,
    max_steps=1000
)

# Experiment

In [22]:
environment.episode()

(1.8283518464963722, 270)

In [23]:
environment.exploration_segment(1)

(0.0, 0.0)

In [24]:
environment.batch(20)

In [25]:
agent.commit()

# Training

In [ ]:
environment.train(segments=100, episodes=100)

  0%|                                                                       | 0/100 [00:00<?, ?it/s]

In [23]:
environment.save('../saved/')